In [6]:
import BeautifulSoup
import selenium
import json

compounds = json.load(open('2021-04-29_APF_complete thermodynamically described compounds for KEGG_01.json'))

num = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

general_delay = 2  #seconds



def click_element_type(n_type):
    global driver
    element = driver.find_element_by_xpath(n_type)
    element.click()
    time.sleep(general_delay)
    
def select_dropdown_id(n_id, n_choice):
    global driver
    element = Select(driver.find_element_by_id(n_id))
    element.select_by_visible_text(n_choice)
    time.sleep(general_delay)

def chunks(data, SIZE):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
        
        

def scrape(compound):
    global driver
    global enzymes
    
    driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
    driver.get("https://www.genome.jp/kegg/")
    
    time.sleep(general_delay)
    
    # search field
    text_area = driver.find_element_by_name("text")
    text_area.send_keys(compound)
    time.sleep(general_delay)
    click_element_id("//input[@value='Search']")
    time.sleep(general_delay)
    
    # display all compound button
    click_element_type("/html/body/form/a[@href='javascript:MoreSearch(\"compound\")']")
    
    # acquire the list of results
    '''possible_compounds = []
    for div in click_element_type("/html/body/form/div[@style='width_600px']"):'''
    url = driver.current_url
    
    
        

    compounds[compound] = "no"
    driver.close()
    return

    
    time.sleep(general_delay)
    
    select_dropdown_id("max", "100")
    element = Select(driver.find_element_by_id("max"))
    element.select_by_visible_text("100")
    
    time.sleep(general_delay)


    else:
        compounds[compound] = "no"
        driver.close()
        return
    
    enzymes[enzyme_name] = "yes"
    
    driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")
    
    time.sleep(15)
    
    click_element_id("excelExport")
    
    time.sleep(5)
    
    driver.close()
    


for compound in compounds:
    if compounds[compound] != "yes" and compounds[compound] != "no":
        flag = True
        while flag:
            try:
                scrape(compound)
                flag = False
            except:
                time.sleep(10)
            
        with open('compounds_to_collect_out.json', 'w') as outfile:
            json.dump(compounds, outfile, indent = 3)   
            outfile.close()



SyntaxError: invalid syntax (<ipython-input-6-dc0e7321c42b>, line 73)

The NIST compounds ar eparsed through the ModelSEED files. The combination of the list of NIST compounds are manually edited to remove illogical names and parsing errors. 

In [2]:
# import libraries and the text files 
import pandas
import json

ids = pandas.read_csv('Unique_ModelSEED_Compound_Aliases.txt', sep = '\t')
names = pandas.read_csv('Unique_ModelSEED_Compound_Names.txt', sep = '\t')
compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))

#display(ids.head(5))
#display(names.head(5))

# parse the compounds through the ModelSEED text files
described_compounds = {}
repeated_compounds = []
for compound in compounds:
    for index, modelseed_name in names.iterrows():
        if str(compound).lower() == str(modelseed_name['External ID']).lower():
            if modelseed_name['External ID'] not in described_compounds:
                described_compounds[modelseed_name['External ID']] = modelseed_name['ModelSEED ID']
                
for described_compound, modelseed_id in described_compounds.items():
    for index2, id in ids.iterrows():
        if id['ModelSEED ID'] == modelseed_id:
            if id['Source'] == 'KEGG':
                #print(modelseed_name)
                #print(modelseed_name['ModelSEED ID'])
                #print(id['External ID'])
                compound_information[described_compound] = {'ModelSEED ID': modelseed_id,
                                                            'KEGG ID':id['External ID']}

compounds_list = []
for compound in compound_information.keys():
    compounds_list.append(compound)
    
print('Identified compounds: ', len(compounds_list))
print('repeated compounds: ', len(repeated_compounds))

'''with open('2021-05-09_APF_KEGG and ModelSEED_IDs_01', 'w') as output:
    json.dump(compound_information, output, indent = 4)'''

Identified compounds:  918
repeated compounds:  0


"with open('2021-05-09_APF_KEGG and ModelSEED_IDs_01', 'w') as output:\n    json.dump(compound_information, output, indent = 4)"

In [52]:
# import the requisite libraries
from ftfy import fix_encoding
from datetime import date
import numpy
import json
import pandas
import re

modelseed = json.load(open('compounds.json'))
compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))

empty_cells = ['null', ' null', numpy.nan, 'None', None, ""]

# parse the modelseed database and the list of thermodynamically described compounds
'''nist_compounds = []
for compound in compounds:
    nist_compounds.append(compound)
nist_compounds_string = ', '.join(nist_compounds)
print(nist_compounds_string)'''

modelseed_compounds = {}
all_names = []
for compound2 in modelseed:
    if compound2['is_obsolete'] == 0:
        try:
            for entry in compound2['aliases']:
                if re.search('Name:', entry):
                    alternative_names = re.sub('Name: ', '', entry).split('; ')

        except:
            alternative_names = []
               
        if re.search('KEGG:', entry):
            kegg_id = re.sub('KEGG: ', '', entry).split('; ')
            
        modelseed_compounds[compound2['abbreviation'].lower()] = {'ModelSEED ID': compound2['id'],
                                                          'KEGG ID': kegg_id,
                                                          'alternative names': alternative_names}
    '''else:
        alternative_names = []'''

    all_names.append([compound2['abbreviation'].lower()] + alternative_names)

#print(modelseed_abbreviations)
total_modelseed_names = []
for name_list in all_names:
    total_modelseed_names += name_list 
total_modelseed_names_string = ',\t'.join(list(set(total_modelseed_names)))   
#    if all(name not in described_compounds.keys() for name in all_names):

modelseed_abbreviations = modelseed_compounds.keys()
modelseed_abbreviations_string = ',\t'.join(modelseed_abbreviations)
if 'd-fructose' in modelseed_abbreviations:
    print('yes')

described_compounds = {}
undescribed_compounds = []
for compound in compounds:         
    compound = compound.lower()
    #print(compound)
    compound = re.sub('\+', '\\+', compound)
    if compound not in described_compounds.keys():
        '''try:'''
        if re.search(compound, total_modelseed_names_string, re.IGNORECASE):
            if re.search(', {}, '.format(compound), modelseed_abbreviations_string, re.IGNORECASE):
                compound = fix_encoding(compound)
                print('described 1: ', compound)
                #re.sub(',\s(\w)\w+', '\U\1', ', {}, '.format(compound))
                described_compounds[compound] = modelseed_compounds[compound]
            else:
                for name in all_names:
                    name_string = ', '.join(name)
                    if re.search('(^|,) {},'.format(compound), name_string, re.IGNORECASE):
                        root_compound = name[0]
                        root_compound = fix_encoding(root_compound)
                        print('described 2: ', root_compound)
                        #print(name_string)
                        described_compounds[root_compound] = modelseed_compounds[root_compound]

        else:
            if compound not in undescribed_compounds:
                compound = fix_encoding(compound)
                undescribed_compounds.append(compound)
                print('{} is undescribed by ModelSEED'.format(compound))

        '''except:
            print('ERROR: The {} compound is incompatible with the regex search function .'.format(compound))'''


# export the acquired set of described compounds
with open('{}_APF_described compounds in ModelSEED_01.json'.format(date.today()), 'w') as output:
    json.dump(described_compounds, output, indent = 5)
    
# export the set of undescribed compounds
with open('{}_APF_undescribed compounds in ModelSEED_01.csv'.format(date.today()), 'w', encoding="utf-8") as output:
    undescribed_dataframe = pandas.DataFrame(undescribed_compounds, columns = ['undescribed compounds'])
    undescribed_dataframe.to_csv(output)

described 2:  asp-l
described 2:  fum
described 2:  nh4
described 2:  h2o
(s)-malate is undescribed by ModelSEED
described 2:  2pg
described 2:  pep
described 2:  fdp
described 2:  dhap
described 2:  3pg
described 2:  argp
described 2:  arg-l
described 2:  octp
described 2:  pi
described 2:  phosphocreatine
described 2:  creatine
(r)-3-phosphoglycerate is undescribed by ModelSEED
(r)-glycerate is undescribed by ModelSEED
described 2:  pyr
described 2:  f1p
described 2:  f6p
described 2:  beta-d-fructose 6-phosphate
described 2:  fru
described 2:  d-fructose, o-beta-d-fructofuranosyl-(2->1)-o-beta-d-fructofuranosyl-(2->1)-
described 2:  d-fructose, 6-o-beta-d-fructofuranosyl-
described 2:  dgal6p
described 2:  gal
described 2:  cll_199
described 2:  g6p
described 2:  glc-d
described 2:  alpha-d-glucose
described 2:  1,6-anhydroglucose
described 2:  2-o-beta-d-glucopyranosyl-beta-d-glucopyranose
described 2:  (indole-3-acetyl)-glucan
l-α-glycerophosphate is undescribed by ModelSEED
descr

n-benzoyl-l-tyrosylglycinanilide is undescribed by ModelSEED
glycylanalide is undescribed by ModelSEED
n-acetyl-l-tyrosine hydroxamic acid is undescribed by ModelSEED
described 2:  hydroxylamine
n-acetyl-l-tryptophan ethyl ester is undescribed by ModelSEED
n-acetyl-l-phenylalanine methyl ester is undescribed by ModelSEED
described 2:  n-acetyl-l-phenylalanine
n-acetyl-l-phenylalanylglycinamide is undescribed by ModelSEED
3-(2-furyl)acryloylimidazole is undescribed by ModelSEED
furylacrylic acid is undescribed by ModelSEED
described 2:  imidazole
described 2:  3-methylhistamine
described 2:  1-(2,4-dichloro-beta-((2,4-dichlorobenzyl)oxy)phenethyl) imidazole
n-acetyl-l-tryptophan phenylethyl ester is undescribed by ModelSEED
described 2:  2phetoh
2-propylhippurate is undescribed by ModelSEED
described 2:  bgly
n-acetyl-l-phenylalanine ethyl ester is undescribed by ModelSEED
described 2:  ala-d
described 2:  raffin
described 2:  stchs
o-α-d-galactopyranosyl-(1->6)-o-α-d-galactopyranosyl-(

described 2:  d-glucitol 6-phosphate
n-acetyl-α-d-glucosamine 1-phosphate is undescribed by ModelSEED
described 2:  uacgam
described 2:  forglu
β-alanylpantetheine is undescribed by ModelSEED
acrylylpantetheine is undescribed by ModelSEED
described 2:  galur
described 2:  tagur
described 2:  glcur
described 2:  fruur
described 2:  acnam
described 2:  acmana
described 2:  neungc
2-keto-3-deoxy-d-glycero-d-galacto-nonopyranulosonate is undescribed by ModelSEED
described 2:  udp-l-arabinose
described 2:  udp-xylose
described 2:  udpglcur
described 2:  udpgalur
described 2:  12ppd-s
described 2:  (s)-propane-1,2-diol
described 2:  1,2-dihydroxypropane
described 2:  lald-l
described 2:  lys-d
described 2:  methf
described 2:  5,10-methenyl-thf
described 2:  10fthf
described 2:  suc6p
described 2:  sucrose 6-phosphate
described 2:  4hdxbutn
described 2:  2dr5p
l-arginosuccinate is undescribed by ModelSEED
described 2:  ara5p
β-d-glucose 6-phosphate is undescribed by ModelSEED
atp \+l-valine 

atp-γ-s is undescribed by ModelSEED
2-thio-5-diphosphomevalonate is undescribed by ModelSEED
described 2:  26dap-ll
described 2:  26dap-m
(3s)-3,6-diaminohexanoate is undescribed by ModelSEED
5-hydroxy-n-methylpyroglutamate is undescribed by ModelSEED
described 2:  mea
4-methyl-2-oxopentanooate is undescribed by ModelSEED
described 2:  l2aadp
described 2:  2oxoadp
β-(3,5-diiodo-4-hydroxyphenyl)lactate is undescribed by ModelSEED
β-(3,5-diiodo-4-hydroxyphenyl)pyruvate is undescribed by ModelSEED
described 2:  2,4-diaminopentanoate
described 2:  2-amino-4-oxopentanoate
described 2:  cytidine
described 2:  7,8-dihydrobiopterin
described 2:  sepiapterin
described 2:  2-methyleneglutarate
described 2:  methylitaconate
1-phospho-α-d-glucuronate is undescribed by ModelSEED
adenosine 2':3'-(cyclic)phosphate is undescribed by ModelSEED
described 2:  3amp
described 2:  3gmp
described 2:  dgmp
described 2:  dgdp
(s)-5-amino-3-oxohexanoate is undescribed by ModelSEED
described 2:  2-oxoglutaramate

described 2:  optal
described 2:  pphn
described 2:  aproa
described 2:  13dampp
described 2:  cholp
described 2:  4hbz
described 2:  phpyr
described 2:  34hpp
described 2:  1,4,5,6-tetrahydro-4,6-dioxo-s-triazine-2-carboxylic acid
dihydroxonic acid is undescribed by ModelSEED


In [69]:
from ftfy import fix_encoding
import pandas
import json
import io

compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))
undescribed_compounds = pandas.read_csv('2021-06-13_APF_undescribed compounds in ModelSEED_01.csv')
described_compounds = json.load(open('2021-06-13_APF_described compounds in ModelSEED_01.json'))
    
print('Described compounds: ', len(described_compounds.keys()))
print('Undescribed compounds: ', len(undescribed_compounds['undescribed compounds']))
print('Total compounds: ', len(compounds.keys()))

compounds_list = []
for compound in compounds:
    compound = compound.lower()
    compounds_list.append(compound)
    
undescribed_compounds_list = []
for compound in undescribed_compounds:
    compound = fix_encoding(compound)
    undescribed_compounds_list.append(compound)
    
modeled_compounds = set(undescribed_compounds_list + list(described_compounds.keys())) 
print('modeled compounds: ', len(modeled_compounds))
missing_compounds = set(compounds_list) - set(undescribed_compounds_list)
missing_compounds -= set(described_compounds.keys())
print('Missing compounds: ', len(missing_compounds))
for compound in missing_compounds:
    print(compound)


    
'''undescribed_compounds = list(set(compounds_list) - set(described_compounds.keys()))
print('Undescribed compounds: {}\n'.format(len(undescribed_compounds)), '='*len('Undescribed compounds:'))
for compound in undescribed_compounds:
    print(compound)'''
    
    
'''with io.open('2021-05-09_APF_NIST compounds absent from ModelSEED_01.csv', 'w', encoding="utf-8") as output:
    undescribed_compund_csv = pandas.DataFrame(undescribed_compounds)
    undescribed_compund_csv.to_csv(output)'''

Described compounds:  541
Undescribed compounds:  351
Total compounds:  903
modeled compounds:  543
Missing compounds:  777
udp-n-acetyl-d-glucosamine
dimethylacetothetin
l-histidine
(r)-2-heptyl butyrate
l-phenylalanylglycine
2-oxobutanoate
d-glucose
dtdp-4-amino-4,6-dideoxy-d-glucose
urocanate
3-(2-furyl)acryloylimidazole
cdpethanolamine
(s)-2-(4-isobutylphenyl)propionyl-coa
glutathione
α-d-melibiose
5β-pregnane-3α,17α,21-triol-11,20-dione
l,l-2,6-diaminoheptanedioate
(3-phospho-d-glycerol-chloromethyl)phosphonate
reduced pyocyanine
3-phosphohydroxypyruvate
phenoxymethylpenicillinoic acid
3-oxobutanoyl-coa
(3-phospho-d-glycerol-chloroethyl)phosphonate
n-acetyl-d-glucosamine 1-phosphate
desamino nad
5,10-methylenetetrahydrofolate
2-oxo-d-gluconate
3'-dephospho-coa
benzaldehyde
dimethylmaleate
(r)-2-dodecyl butyrate
2-methyl-3-oxopropanoate
7-aminodeacetoxycephalosporanic acid
laminaritetraose
methyl-(e)-2-octenoate
udpgalactose
l-aspartate
2'-deoxyadenosine
10/20-dna oligomer
methyler

In [48]:
from ftfy import fix_encoding
import json

nist = json.load(open('2021-05-06_NIST consolidated.json'))
described_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))

described_compounds_list = []
for compound in described_compounds:
    described_compounds_list.append(compound)

#print('Described compounds: {}\n'.format(len(described_compounds_list)), described_compounds_list)
    
completely_described_reactions = {}
undescribed_reactions = {}
printed_enzyme = 'no'
for enzyme, information in nist.items():
    described_reactions = []
    for reaction in information['reaction']:
        reaction = fix_encoding(reaction)
        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            compounds = reactants_list + products_list
            for element in compounds:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element not in compounds_list:
                    compounds_list.append(element)

        #print(compounds_list)
        described = all(compound in described_compounds_list for compound in compounds_list)
        if described:
            described_reactions.append(reaction)

        '''else:
            undescribed_reactions[element] = {'Enzyme': enzyme,
                                              'Reaction': reaction,
                                              'synonym': ''}
        
            continue'''
        
    completely_described_reactions[enzyme] = {'completely described reactions': described_reactions}
    
with open('2021-05-11_APF_completely described reactions for ModelSEED_01.json', 'w') as output:
    json.dump(completely_described_reactions, output, indent = 4)

In [69]:
import json

def isfloat(number):
    try:
        float(number)
        return True
    except ValueError:
        return False

completely_described_reactions = json.load(open('2021-05-09_APF_completely described reactions for ModelSEED_01.json'))
nist = json.load(open('2021-05-06_NIST consolidated.json'))

described_enzymes_list = []
for enzyme in completely_described_reactions:
    described_enzymes_list.append(enzyme)
    
described_enzyme_data = {}
for enzyme, information in nist.items():
    unique_reactions = []
    temperatures = []
    phs = []
    if enzyme in described_enzymes_list:
        for reaction in information['reaction']:
            if reaction not in unique_reactions:
                unique_reactions.append(reaction)
        
        # calculate the average temperature 
        for temperature in information['experimental temperatures']:
            if isfloat(temperature):
                temperatures.append(float(temperature))
        
        if len(temperatures) > 0:
            average_temperature = sum(temperatures) / len(temperatures)
        else:
            average_temperautre = 'nan'
            
        # calculate the average ph 
        for ph in information['experimental phs']:
            if isfloat(ph):
                phs.append(float(ph))
        
        if len(temperatures) > 0:
            average_ph = sum(phs) / len(phs)
        else:
            average_ph = 'nan'
        

        
    described_enzyme_data[enzyme] = {'unique reactions': unique_reactions,
                                    'average keq': information['Keq']['keq average'],
                                    'average enthalpy': information['Enthalpy']['enthalpy average'],
                                    'average temperature kelvin': average_temperature,
                                    'average pH': average_ph}

with open('2021-05-09_APF_ModelSEED described enzymes_01.json', 'w') as output:
    json.dump(described_enzyme_data, output, indent = 4)

In [55]:
print('\xb0K')

°K


The reactions are partitioned into a TSV file. The file information is consistent with the reaction example https://github.com/ModelSEED/ModelSEEDDatabase/blob/chemistry_staging/Biochemistry/Curation/janakagithub/CentralMetabolism_InorganicMetabolism.tsv. The equation for each reaction which contains the ModelSEED IDs must be acquired and associated with each enzyme for the TSV output. 

In [ ]:
import json

nist_data = json.load(open('2021-05-09_APF_ModelSEED described enzymes_01.json'))

     
with open('2021-05-10_APF_ModelSEED formatted reactions.tsv', 'w') as output:
    output.write('\t'.join(['ID', 'Names','Equation', 'Equation defintion', '\n']))
    for enzyme, information in nist_data.items():
        for reaction in information['unique reactions']:            
            output.write('\t'.join([information['id'],
                                    enzyme,
                                    equation,
                                    reaction) + '\n')
    output.close()
                                    
        

The compound names for the set of completely described NIST reactions will be exchanged with the respective ModelSEED IDs. The exchanged reaction strings will be exported and then processed to form the reaction TSV file that will be passed through the ModelSEED database codes to contribute the reaction information. 

In [ ]:
from ftfy import fix_encoding
import json

nist_reactions = json.load(open('2021-05-09_APF_ModelSEED described enzymes_01.json'))
nist_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))

for enzyme, information in nist_reactions.items():
    for reaction in information['unique reactions']:        
        reaction = fix_encoding(reaction)
        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reaction_string = ''
            for element in reactants_list:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    element = nist_compounds[element]['ModelSEED ID']
                    reaction_string += element
                    
            product_string = ''
            for element in products_list:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    element = nist_compounds[element]['ModelSEED ID']
                    
                    
with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)

The ModelSEED IDs were substituted into the reaction strings. The set of substituted reactiosn for each enzyme was subsequently exported as a JSON file. 

In [68]:
from ftfy import fix_encoding
import json
import re

nist_reactions = json.load(open('2021-05-09_APF_ModelSEED described enzymes_01.json'))
nist_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))
described_reactions = json.load(open('2021-05-09_APF_nist thermodynamically described reactions for ModelSEED_01.json'))

total_complete_reactions = []
exchanged_reactions = []
to_modelseed_reactions = {}
for enzyme, information in nist_reactions.items():
    #if enzyme in described_reactions.keys():
    #print('\n\n{}\n'.format(enzyme))
    #for modelseed_id in information['ModelSEED ID']:
    complete_reactions = []
    for reaction in information['unique reactions']:
        reaction = fix_encoding(reaction)
        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)
        #print('reaction: {}'.format(reaction))

        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reactants = []
            for element in reactants_list:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    reactants.append(nist_compounds[element]['ModelSEED ID'])

                else:
                    reactants.append(element)

            products = []
            for element in products_list:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    products.append(nist_compounds[element]['ModelSEED ID'])

                else:
                    products.append(element)

            compounds = reactants + products
        
        else:
            print('incorrect reaction: ', reaction)

        if len(reactants) > 1:
            reactants_string = ' + '.join(reactants)
            #print('yes')

        elif len(reactants) == 1:
            reactants_string = reactants[0]

        else:
            print('ERROR: The reaction __ {} __ has zero reactants'.format(reaction))

        #print(products)
        if len(products) > 1:
            products_string = ' + '.join(products)

        elif len(products) == 1:
            products_string = products[0]

        else:
            print('\tERROR: The reaction __ {} __ has zero products'.format(reaction))
            print('\t{}'.format(products_list))

        #print('''reaction_string: {}
        #products_string: {}'''.format(type(reaction_string), type(products_string)))

        final_reaction = reactants_string + ' = ' + products_string
        #print('exchanged reaction: {}'.format(final_reaction))
        exchanged_reactions.append(final_reaction)

        if all('cpd' in compound for compound in compounds):
            complete_reactions.append(final_reaction)

        
        '''for compound, ids in nist_compounds.items():
            if re.search(compound, reaction, re.IGNORECASE):
                if not re.search('\w {}'.format(compound), reaction, re.IGNORECASE):
                reaction = re.sub(compound, ids['ModelSEED ID'], reaction)'''

    to_modelseed_reactions[enzyme] = complete_reactions
    total_complete_reactions += complete_reactions
    
print('Total reactions: {}'.format(len(exchanged_reactions)))                    
print('Parsed and exchaged reactions: {}'.format(len(total_complete_reactions)))
print('Previously complete reactions: {}'.format(len(described_reactions.keys())))
#print(complete_reactions)


'''with open('2021-05-11_APF_NIST to ModelSEED_01.json', 'w') as output:
    json.dump(to_modelseed_reactions, output, indent = 4)'''


'''with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)'''

incorrect reaction:   cleavage of a single bond in casein  
Total reactions: 774
Parsed and exchaged reactions: 396
Previously complete reactions: 326


"with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:\n    json.dump( , output, indent = 4)"

The substituted reactions will be formalized into a TSV file that will be passed through the ModelSEED database. 

In [93]:
import json
import re

described_reactions = json.load(open('2021-05-11_APF_NIST to ModelSEED, in form_01.json'))

tsv_lines = []
for enzyme, information in described_reactions.items():
    for reaction in information:
        tsv_lines.append('\t'.join([enzyme, reaction]))
        
print('Total reactions: {}'.format(len(tsv_lines)))  
        
tsv_file = open('2021-05-11_APF_NIST to ModelSEED, per reaction, in form_01.tsv', 'w')
tsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))
tsv_file.close()    

'''with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)'''

Total reactions: 396


"with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:\n    json.dump( , output, indent = 4)"

The reactions were completely prepared for passage through the ModelSEED code. The compounds were all substituted with the ModelSEED IDs and the reaction string format was aligned with the ModelSEED conventions. The TSV file will be exported and then executed through the ModelSEED Add_new_curated_reactions.py file. 

In [100]:

from ftfy import fix_encoding
import json
import re

nist_reactions = json.load(open('2021-05-09_APF_ModelSEED described enzymes_01.json'))
nist_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))
described_reactions = json.load(open('2021-05-09_APF_nist thermodynamically described reactions for ModelSEED_01.json'))

total_complete_reactions = []
exchanged_reactions = []
to_modelseed_reactions = {}
for enzyme, information in nist_reactions.items():
    #if enzyme in described_reactions.keys():
    #print('\n\n{}\n'.format(enzyme))
    #for modelseed_id in information['ModelSEED ID']:
    complete_reactions = []
    for reaction in information['unique reactions']:
        reaction = fix_encoding(reaction)
        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)
        #print('reaction: {}'.format(reaction))

        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reactants = []
            for element in reactants_list:
                #print(element)
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = re.compile('(\d\s|\d\/\d\s)')
                    coefficient = coefficient.findall(element)[0]
                else:
                    coefficient = ''
                    
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    reactants.append(coefficient + nist_compounds[element]['ModelSEED ID'])

                else:
                    reactants.append(element)

            products = []
            for element in products_list:
                #print(element)
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = re.compile('(\d\s|\d\/\d\s)')
                    coefficient = coefficient.findall(element)[0]
                else:
                    coefficient = ''
                    
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    products.append(coefficient + nist_compounds[element]['ModelSEED ID'])

                else:
                    products.append(element)

            compounds = reactants + products
            
        
        else:
            print('incorrect reaction: ', reaction)

        if len(reactants) > 1:
            reactants_string = ' + '.join(reactants)
            #print('yes')

        elif len(reactants) == 1:
            reactants_string = reactants[0]

        else:
            print('ERROR: The reaction __ {} __ has zero reactants'.format(reaction))

        #print(products)
        if len(products) > 1:
            products_string = ' + '.join(products)

        elif len(products) == 1:
            products_string = products[0]

        else:
            print('\tERROR: The reaction __ {} __ has zero products'.format(reaction))
            print('\t{}'.format(products_list))

        reaction_string = reactants_string + ' <=> ' + products_string
            
        # arrange the reaction equilibrium arrow and stoichiometric coefficients for ModelSEED                        
        if re.search('((?<!\w)\d\s|\d\/\d\s)', reaction_string):
            reaction_string = re.sub('((?<!\w)\d\s|\d\/\d\s)', r'(\1) ', reaction_string)
            reaction_string = re.sub('(?<=\d)(\s)(?=\))', '', reaction_string)
            
        if re.search('(?<=>|\+)(\s)(?=\w)', reaction_string):
            reaction_string = re.sub('(?<=>|\+)(\s)(?=\w)', ' (1) ', reaction_string)

        if not re.search('^( \d|\()', reaction_string):
            reaction_string = '(1) ' + reaction_string
            
        reaction_string = re.sub('(?<=\w)(\s)(?=<|\+)', '[0] ', reaction_string)
        reaction_string += '[0]'
            
        #print('''reaction_string: {}
        #products_string: {}'''.format(type(reaction_string), type(products_string)))
        #print('exchanged reaction: {}'.format(final_reaction))
        
        
        exchanged_reactions.append(reaction_string)
        if all('cpd' in compound for compound in compounds):
            complete_reactions.append(reaction_string)

        
        '''for compound, ids in nist_compounds.items():
            if re.search(compound, reaction, re.IGNORECASE):
                if not re.search('\w {}'.format(compound), reaction, re.IGNORECASE):
                reaction = re.sub(compound, ids['ModelSEED ID'], reaction)'''

    to_modelseed_reactions[enzyme] = complete_reactions
    total_complete_reactions += complete_reactions
    
print('Total reactions: {}'.format(len(exchanged_reactions)))                    
print('Parsed and exchaged reactions: {}'.format(len(total_complete_reactions)))
print('Previously complete reactions: {}'.format(len(described_reactions.keys())))
#print(complete_reactions)


'''with open('2021-05-11_APF_NIST to ModelSEED, in form_01.json', 'w') as output:
    json.dump(to_modelseed_reactions, output, indent = 4)'''

    
tsv_lines = []
enzyme_id_dictionary = {}
id_number = 0
for enzyme, information in to_modelseed_reactions.items():
    enzyme_id_dictionary[enzyme] = id
    for reaction in information:
        id = 'rxn{}'.format(id_number)
        tsv_lines.append('\t'.join([id, reaction]))
        id_number += 1
    
print('Total reactions: {}'.format(len(tsv_lines)))  
        
tsv_file = open('2021-05-11_APF_NIST to ModelSEED, in form, with ids_01.tsv', 'w')
tsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))
tsv_file.close()  
    

'''with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)'''

incorrect reaction:   cleavage of a single bond in casein  
Total reactions: 774
Parsed and exchaged reactions: 396
Previously complete reactions: 326
Total reactions: 396


"with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:\n    json.dump( , output, indent = 4)"

In [29]:
import re

compound = 'hi, yes, no'
re.sub('(^|, )(\S+)\w', '\u\1', ', {}, '.format(compound))
print(compound)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape (<ipython-input-29-f8fcd23c4c81>, line 4)